In [1]:
#!pip install wrapt

In [2]:
import os
import logging
import sys
import contextlib
from pprint import pprint
import wrapt  
import functools


# Задача 1 Применение метаклассов.

Идея решения.
В `__new__()` метакласса AttrLoggingMeta:
* методы и аттрибуты начинающиеся с '__' пропускаются. Предполагаем, что приватные методы не требуется логгировать.
* методы оборачиваются декоратором log_access() - статическим методом класса AttrLoggingMeta
* для аттрибутов:
    * создается аттрибут класса с именем по масске `f'__logged_by_ALG_{исходное имя аттрибута}'`
    * сам же атрибут подменяется на экземпляр класса property(), для которого геттер и сеттер созданы с логгированием

Особенности и ограничения:
* аттрибуты экземпляра класса (добавляемые в `__init__()`) логгировать не будут, т.к. появляются после того, как отработает `__new__()` метакласса

In [3]:
class AttrLoggingMeta(type):
    """Мета-класс с контролем доступа к методам и чтения-записи атрибутов"""
    def __new__(mcs, name, bases, attrs, **extra_kwargs):

        need_change_attr = []

        for attr, method in attrs.items():  
            if not attr.startswith('__'): # пропустим все приватные аттрибуты и методы
                if callable(method):
                    # оборачиваем все методы декоратором-логгером
                    attrs[attr] = AttrLoggingMeta.log_access(method) 
                else:
                    # запомним аттрибуты
                    need_change_attr.append(attr)
                
        # для каждого аттрибута создадим property со специальными сеттером и геттором
        for attr in need_change_attr:
            attrs[f'__logged_by_ALG_{attr}'] = attrs[attr]
            attrs[attr] = property(fget=AttrLoggingMeta.__create_getter(attr), fset=AttrLoggingMeta.__create_setter(attr))

        cls_obj = super().__new__(mcs, name, bases, attrs)  
        return cls_obj

    def __init__(cls, name, bases, attrs, **extra_kwargs):  
        super().__init__(cls)  

    @classmethod  
    def __prepare__(mcs, cls, bases, **extra_kwargs):  
        return super().__prepare__(mcs, cls, bases, **extra_kwargs)  
        
    @staticmethod
    def log_access(func):
        """Логгер доступа к методу"""
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(f"AttrLoggingMeta: Calling method {func.__qualname__}()")
            return func(*args, **kwargs)
        return wrapper

    @staticmethod
    def __create_getter(name):
        """Создание геттера-логгера"""
        body_getter =f"""
def __getter(self):
    print(f"AttrLoggingMeta: Reading attribute: {name}")
    return self.__logged_by_ALG_{name}
        """
        #print(body_getter)
        exec(body_getter)
        return locals()[f'__getter']

    @staticmethod
    def __create_setter(name):
        """Создание сеттера-логгера"""
        body_setter =f"""
def __setter(self, value):
    print(f"AttrLoggingMeta: Writing attribute: {name} with value {{value}}")
    self.__logged_by_ALG_{name} = value
        """
        #print(body_setter)
        exec(body_setter)
        return locals()[f'__setter']

    def __call__(cls, *args, **kwargs):  
        return super().__call__(*args, **kwargs)    


In [4]:
class LoggedClass(metaclass = AttrLoggingMeta): 
    """Тестовый класс"""
    class_attr = "class_attr_value"
    
    def __new__(cls, attr1, attr2, attr3):  
        return super().__new__(cls)  
        
    def __init__(self, attr1, attr2, attr3):
        self.instance_attr1 = attr1
        self.__attr2 = attr2
        self.__attr3 = attr3

    def my_method(self):
        return('LoggedClass.my_method() is runned()')

    def set_instance_attr1(self, value):
        self.instance_attr1 = value
        print('LoggedClass.set_instance_attr1() is runned()')
    
    # read-only property класса
    @property
    def class_attr2(self):
        print('LoggedClass.attr2 is read()')
        return self.__attr2
    
    # mutable property класса
    @property 
    def class_attr3(self):
        print('LoggedClass.attr3 is read()')
        # значение оборачивается в звездочки, чтобы убедиться, что заданный в классе геттер работает
        return f'**{self.__attr3}**'
    
    @class_attr3.setter
    def class_attr3(self, value):
        print('LoggedClass.attr3 is write()')
        # значение свойства "удваивается", чтобы проверить, что заданный в классе сеттер работает
        self.__attr3 = f'{value}_{value}'
    
lc = LoggedClass(attr1="attr1_value", attr2="attr2_value", attr3 = "attr3_value")    

### Проверить логгирование обращений к методам

In [5]:
# Проверяем логгирование возвращающего метода
print(lc.my_method())


AttrLoggingMeta: Calling method LoggedClass.my_method()
LoggedClass.my_method() is runned()


In [6]:
# Проверяем логгирование метода без возвращаемого значения, но с параметором
print("lc.instance_attr1 = " + lc.instance_attr1)
lc.set_instance_attr1("new_instance_attr1_value")
print("lc.instance_attr1 = " + lc.instance_attr1)

lc.instance_attr1 = attr1_value
AttrLoggingMeta: Calling method LoggedClass.set_instance_attr1()
LoggedClass.set_instance_attr1() is runned()
lc.instance_attr1 = new_instance_attr1_value


### Проверить логгирование обращений к class_attr

In [7]:
# чтение
print(f'lc.class_attr={lc.class_attr}')

AttrLoggingMeta: Reading attribute: class_attr
lc.class_attr=class_attr_value


In [8]:
# изменение
lc.class_attr = "class_attr_new"


AttrLoggingMeta: Writing attribute: class_attr with value class_attr_new


In [9]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc.class_attr={lc.class_attr}')


AttrLoggingMeta: Reading attribute: class_attr
lc.class_attr=class_attr_new


### Проверить логгирование обращений к instance_attr1

Логов быть не должно, т.к. это не аттрибут класса, а атрибут экземпляра

In [10]:
# чтение
print(f'lc.instance_attr1={lc.instance_attr1}')

lc.instance_attr1=new_instance_attr1_value


In [11]:
# изменение
lc.instance_attr1 = "instance_attr1_new"

In [12]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc.instance_attr1={lc.instance_attr1}\n')

lc.instance_attr1=instance_attr1_new



### Проверить логгирование обращений к class_attr2

In [13]:
# чтение
print(f'lc.class_attr2={lc.class_attr2}')

AttrLoggingMeta: Reading attribute: class_attr2
LoggedClass.attr2 is read()
lc.class_attr2=attr2_value


In [14]:
# Изменение. Д.б. ошибка, т.к. у свойста не задан сеттер
try:
    lc.class_attr2 = "class_attr2_new"
except Exception as e:
   print(f'Ошибка: {e}')

AttrLoggingMeta: Writing attribute: class_attr2 with value class_attr2_new
Ошибка: property '__logged_by_ALG_class_attr2' of 'LoggedClass' object has no setter


In [15]:
### Проверить логгирование обращений к class_attr3

In [16]:
# чтение
print(f'lc.class_attr3={lc.class_attr3}')

AttrLoggingMeta: Reading attribute: class_attr3
LoggedClass.attr3 is read()
lc.class_attr3=**attr3_value**


In [17]:
# изменение
lc.class_attr3 = "class_attr3_new"

AttrLoggingMeta: Writing attribute: class_attr3 with value class_attr3_new
LoggedClass.attr3 is write()


In [18]:
# повторное чтение, чтобы проверить, что изменение сработало
print(f'lc.class_attr3={lc.class_attr3}')

AttrLoggingMeta: Reading attribute: class_attr3
LoggedClass.attr3 is read()
lc.class_attr3=**class_attr3_new_class_attr3_new**


In [19]:
LoggedClass.__dict__

mappingproxy({'__module__': '__main__',
              '__doc__': 'Тестовый класс',
              'class_attr': <property at 0x7bc27c6af3d0>,
              '__new__': <staticmethod(<function LoggedClass.__new__ at 0x7bc27c8594e0>)>,
              '__init__': <function __main__.LoggedClass.__init__(self, attr1, attr2, attr3)>,
              'my_method': <function __main__.LoggedClass.my_method(self)>,
              'set_instance_attr1': <function __main__.LoggedClass.set_instance_attr1(self, value)>,
              'class_attr2': <property at 0x7bc27c6af4c0>,
              'class_attr3': <property at 0x7bc27c6af5b0>,
              '__logged_by_ALG_class_attr': 'class_attr_value',
              '__logged_by_ALG_class_attr2': <property at 0x7bc27ddd2520>,
              '__logged_by_ALG_class_attr3': <property at 0x7bc27c6ae2a0>,
              '__dict__': <attribute '__dict__' of 'LoggedClass' objects>,
              '__weakref__': <attribute '__weakref__' of 'LoggedClass' objects>})

In [20]:
lc.__dict__

{'instance_attr1': 'instance_attr1_new',
 '_LoggedClass__attr2': 'attr2_value',
 '_LoggedClass__attr3': 'class_attr3_new_class_attr3_new',
 '__logged_by_ALG_class_attr': 'class_attr_new'}

# Задача 2 Динамическое создание класса


In [21]:

def  create_class_with_methods(name, attributes, methods):
    # объеденить методы и атрибуты в однин словарь
    new_dict = attributes | methods
    # создать новый класс
    new_class = type(name, (), new_dict)
    return new_class

attributes = { 'species': 'Human', 'age': 25 }
methods = { 'greet': lambda self: f"Hello, I am a {self.species} and I am {self.age} years old." }
DynamicClass = create_class_with_methods('DynamicClass', attributes, methods) 

instance = DynamicClass()
print(instance.greet())

Hello, I am a Human and I am 25 years old.


# Задача 3 Генерация кода


In [22]:
def generate_complex_function(function_name, parameters, function_body):
    """Создать функцию по описанию и вернуть её"""
    # собрать строку с параметрами
    parameters_str = ','.join(parameters)
    # добавить в строки с телом функции отступы
    intendent_function_body = '\n'.join(['    '+line for line in function_body.split('\n')])
    # собрать строку для формирования функции
    exec_str = f"def {function_name}({parameters_str}):\n{intendent_function_body}"
    # создать функцию
    exec(exec_str)
    # извлечь функцию из локальных переменных и вернуть её
    return locals()[function_name]

In [23]:
function_name = 'complex_function'
parameters = ['x', 'y']
function_body = """
if x > y:
   return x - y
else:
   return y - x
"""
complex_func = generate_complex_function(function_name, parameters, function_body)

print(complex_func(10, 5))
print(complex_func(5, 10))

5
5


# 